In [7]:
import gym
env_name = 'CartPole-v0'
env = gym.make(env_name)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [8]:
from tensorflow.python.ops import variables
from tensorflow.python.framework import ops

def rescale_to_normal(array):
    return (array - np.mean(array))/ np.std(array)
        
class EL_optimize(object):
    def __init__(self, sigma = 1, alpha = 1e-3,mini_batch_size = 50):
        self.alpha = tf.constant(alpha)
        self.sigma = tf.constant(sigma)
        self.mini_batch_size = mini_batch_size
        self.current_ind = tf.placeholder("int32")
        self.update_weights = tf.placeholder("float32",shape = [mini_batch_size])
        
        self.set_links_to_vars()

    def set_links_to_vars(self):
        self.noise_vars = []
        self.new_noise_vals_ops = []
        self.set_noise_ops = []
        self.copy_state_ops = []
        self.reset_state_ops = []
        self.update_based_on_reward_ops = []
        
        self.vars = variables.trainable_variables() + ops.get_collection(ops.GraphKeys.TRAINABLE_RESOURCE_VARIABLES)
        for var in self.vars:
            tmp_noise_var = tf.random_normal([np.int(self.mini_batch_size/2)] + var.get_shape().as_list())
            noise_var = tf.concat([tmp_noise_var,-tmp_noise_var],0)
            current_noise_vals = tf.Variable(tf.zeros(noise_var.get_shape()))
            self.new_noise_vals_ops.append(current_noise_vals.assign(noise_var))
            self.noise_vars.append(current_noise_vals)
            copy_var = tf.Variable(var.initialized_value())
            
            self.set_noise_ops.append(var.assign(copy_var + self.sigma * current_noise_vals[self.current_ind]))
            self.copy_state_ops.append(copy_var.assign(var))
            self.reset_state_ops.append(var.assign(copy_var))
            
            self.update_based_on_reward_ops.append(var.assign(copy_var + tf.tensordot(self.update_weights, current_noise_vals,axes=[0,0])))
            
    def calc_update_based_on_reward(self,rewards):
        
        self.calc_weights = self.alpha.eval()/(self.mini_batch_size * self.sigma.eval()) * rescale_to_normal(rewards)
    
    def calc_update_based_on_highest_reward(self,rewards):
        
        self.calc_weights = self.sigma.eval()*np.array([(1.0 if val == np.max(rewards) else 0) for val in np.array(rewards)])
         

In [9]:
env.action_space.dtype

dtype('int64')

In [10]:
import tensorflow as tf
import tflearn
import numpy as np
tf.reset_default_graph() # THIS IS NECESSARY BEFORE MAKING NEW SESSION TO STOP IT ERRORING!!
try:
    sess
except:
    pass
else:
    sess.close()
    del sess
sess = tf.InteractiveSession()

n_inputs = env.observation_space.shape[0]
n_hidden = 10  # it's a simple task, we don't need more than this
n_hlayers = 2
n_outputs = env.action_space.n # only outputs the probability of accelerating left
initializer = tf.contrib.layers.variance_scaling_initializer()

# 2. Build the neural network
X = tf.placeholder(tf.float32, shape=[None, n_inputs])
Y = tf.placeholder(tf.int32, shape=[None])

layer = X
for _ in range(n_hlayers):
    layer = tf.layers.dense(layer, n_hidden, activation=tf.nn.relu,
                         kernel_initializer=initializer)

logits = tf.layers.dense(layer, n_outputs,
                          kernel_initializer=initializer)
action = tf.argmax(logits, 1)
    
saver = tf.train.Saver()

el = EL_optimize(alpha = 0.05,sigma = 0.1,mini_batch_size=50)
el.set_links_to_vars()
tf.global_variables_initializer().run()



In [11]:
observation = env.reset()
logs, act = sess.run([logits,action], feed_dict={X: np.expand_dims(observation,axis=0)})
print(logs)
print(act[0])
observation, reward, done, info = env.step(act[0])
print(observation)
print(reward)

[[-0.02009588 -0.02109957]]
0
[ 0.04335854 -0.21724682 -0.0168961   0.30061511]
1.0


In [12]:
env = gym.make(env_name)

render = False

i = 0
max_t = 400

# sess.run(el.alpha.assign(0.01))

while i < 20:
    sess.run(el.copy_state_ops)
    sess.run(el.new_noise_vals_ops)

    rewards_t = np.zeros([el.mini_batch_size])
    obs_std = np.zeros([el.mini_batch_size,env.observation_space.shape[0]])
    
    for j in range(el.mini_batch_size):
        observation = env.reset()
        obs_t = np.array([observation])
        sess.run(el.set_noise_ops, feed_dict={el.current_ind : j})
        reward_t = 0
        for t in range(max_t):
            if render:
                env.render()
            obs_t = np.append(obs_t,[observation],axis = 0)
            
            act = sess.run(action, feed_dict={X: np.expand_dims(observation,axis=0)})[0]
            observation, reward, done, info = env.step(act)
            reward_t += reward
            if done or (t == max_t-1):
                obs_std[j,:] = np.std(obs_t,axis=0)
                rewards_t[j] = reward_t
                break
    print('mini batch results')
        
    if np.std(rewards_t) == 0:
        print('No reward variance, adding obs std as reward')
        rewards_t = np.mean(np.array([(obs) for obs in obs_std.T]),axis=0)

    print(np.mean(rewards_t))
    
#     el.calc_update_based_on_reward(rewards_t)
    el.calc_update_based_on_reward(rewards_t)
    
#     print(el.calc_weights)
    sess.run(el.update_based_on_reward_ops, feed_dict={el.update_weights : el.calc_weights})
        
    i += 1
                
            
# saver.save(sess, "./my_policy_net_basic.ckpt")

env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
mini batch results
18.04
mini batch results
51.64
mini batch results
70.38
mini batch results
27.18
mini batch results
100.84
mini batch results
101.98
mini batch results
55.32
mini batch results
117.06
mini batch results
55.2


KeyboardInterrupt: 

In [ ]:
frames = []
rewardsum = 0
env = gym.make("CartPole-v0").env
obs = env.reset()
with tf.Session() as sess:
    saver.restore(sess, "./my_policy_net_basic.ckpt")
    for step in range(1000):
        env.render(obs)
        action_val = action.eval(feed_dict={X: obs.reshape(1, n_inputs)})
        obs, reward, done, info = env.step(action_val[0])
        rewardsum += reward
        if done:
            break
env.close()
print(rewardsum)
